Import all necessary librarys

In [ ]:
import numpy as np
import sys, os
import time
import matplotlib.pyplot as plt
import sqlite3 as sql

%matplotlib qt

print(os.getcwd())
os.chdir(r'd:\Documenten\Documents\GitHub\Active-Vibration-Stabilization')

In [118]:
DISTANCES = {
'gnx':1.3750, 'gex':0.0000, 'gtx':0.0559,
'gny':0.0000, 'gey':1.3750, 'gty':0.3449,
'gnz':0.7528, 'gez':0.7528, 'gtz':3.2772,
}

AMAT = np.matrix([
#    x | y | z |        rx         |        ry         |          rz
    [0 , 1 , 0 , -DISTANCES['gnz'] ,                 0 , -DISTANCES['gnx']],    # northY
    [0 , 0 , 1 , -DISTANCES['gny'] ,  DISTANCES['gnx'] ,                 0],    # northZ
    [1 , 0 , 0 ,                 0 ,  DISTANCES['gez'] ,  DISTANCES['gey']],    # eastX
    [0 , 0 , 1 , -DISTANCES['gey'] ,  DISTANCES['gex'] ,                 0],    # eastZ
    [1 , 0 , 0 ,                 0 , -DISTANCES['gtz'] ,  DISTANCES['gty']],    # topX
    [0 , 1 , 0 ,  DISTANCES['gtz'] ,                 0 , -DISTANCES['gtx']],    # topY
])
AMAT_INV = np.linalg.inv(AMAT)


In [ ]:
f = .00
file = f'Meetfiles/data/2023-04-15/2023-04-15 - 000959 ({f:.2f})'
db = sql.connect(f'{file}.db')
cur = db.cursor()

timestamps = np.load(f'{file}.npy')




print(timestamps)

# for m,n,o in zip(range(4,len(timestamps),3),range(3,len(timestamps),3),range(2,len(timestamps),3)):
#     print((timestamps[m]-timestamps[n])*timestamps[o])

timestamp = np.where(np.round(timestamps,2) == f)[0][0]
t_min = 0# timestamps[timestamp+1]
t_max = timestamps[timestamp+1] #timestamps[timestamp+2]

print(f'f\t=\t{timestamps[timestamp]}\nt_min\t=\t{t_min}\nt_max\t=\t{ t_max}')

total = np.transpose(cur.execute(f'SELECT * FROM guralp WHERE t BETWEEN {t_min} AND {t_max}').fetchall())


fig1, fig1_axes = plt.subplots(9,1,sharex=True,sharey=True, gridspec_kw=dict(hspace=0))
fig1_manager = plt.get_current_fig_manager()
fig1_manager.window.showMaximized()
fig1_manager.set_window_title('Raw data')


for i, (ax,l) in enumerate(zip(fig1_axes,['topx','topy','topz','northx','northy','northz','eastx','easty','eastz',]),1):
    ax.plot(total[0],total[i])
    ax.set_ylabel(l)
    ax.margins(x= 0)

fig1_axes[0].set_title('raw data', dict(fontsize=48))
plt.pause(0.002)
plt.tight_layout()


In [ ]:
data = np.transpose(cur.execute(f'SELECT t,northY,northZ,eastX,eastZ,topX,topY FROM guralp WHERE t BETWEEN {t_min} AND {t_max} ').fetchall())


In [ ]:
meas = np.mat(data[1:])

out = AMAT_INV*meas
out = np.array(out)


fig2, fig2_axes = plt.subplots(6,1,sharex=True,sharey=True, gridspec_kw=dict(hspace=0))
fig2_manager = plt.get_current_fig_manager()
fig2_manager.window.showMaximized()
fig2_manager.set_window_title('6 DoF measurements')


for i, (ax,l) in enumerate(zip(fig2_axes,['x','y','z','rx','ry','rz'])):
    ax.plot(data[0],out[i])
    ax.set_ylabel(l)
    ax.margins(x= 0)

fig2_axes[0].set_title('6-DoF measurements',dict(fontsize=48))

plt.pause(0.002)
plt.tight_layout()

In [ ]:
dt = np.average([h-l for h,l in zip(data[0][1:],data[0])])
print(data[0,-1]-data[0,0])
fourier_transform = np.fft.fft(data[2], len(data[2]))/len(data[2])
# fourier_transform = fourier_transform[range(int(len(fourier_transform)/2))]
fourier_frequencies = np.fft.fftfreq(len(data[2]),dt)
print(len(data[2]))
print(dt)
tpCount = 718573#len(data[2])
values = np.arange(int(tpCount/2))
timePeriod = tpCount/1000
frequencies = values / timePeriod

fig3, fig3_axes = plt.subplots(1,1,sharex=True,gridspec_kw=dict(hspace=0))
fig3_manager = plt.get_current_fig_manager()
fig3_manager.set_window_title(f'{f:.2f}')
fig3_manager.window.showMaximized()


fig3_axes.set_xscale('log')
fig3_axes.set_yscale('linear')

fig3_axes.plot(fourier_frequencies, np.abs(fourier_transform))

fig3_axes.set_title('Fourier transform of the signal along the z-axis', dict(fontsize=48))
plt.pause(0.002)
plt.tight_layout()

In [ ]:

max_amp = np.max(np.abs(fourier_transform[(0.1 < fourier_frequencies) * (fourier_frequencies < 10)]))

Hz1_4 = np.where(np.abs(fourier_transform) == max_amp)[0][0]

freq = fourier_frequencies[Hz1_4]
amp = np.abs(fourier_transform[Hz1_4])
pha = np.rad2deg(np.angle(fourier_transform[Hz1_4]))
label = fr'''f={freq:.4f}[Hz] A = {amp:.2f} varphi = {pha:.2f} [rad]'''
print(label)
fig3_axes.annotate(label,(freq,amp), textcoords="offset points", xytext=(30,10), ha='center')
fig3_axes.scatter(freq,amp, c='r')


In [ ]:
for num in plt.get_fignums():
    curfig = plt.figure(num)
    curfigmanager = plt.get_current_fig_manager()
    title = f'measure (2 - {curfigmanager.get_window_title()}) Fourier_Transform_z_axis'
    curfig.savefig(f'Meetfiles/data/2023-04-15/{title}.png', transparent= True, dpi = 600, format= 'png')

In [ ]:
db.close()

In [88]:
tf = np.load('TransferFunction_V2.npy')
transferfunction = np.empty((0,4))
for i,trans in zip(np.arange(00.5,3.05,0.05),tf):
    Hz1_4 = i*np.ones_like(fourier_frequencies)
    fr = np.argmin(np.abs(Hz1_4-fourier_frequencies))
    transferfunction= np.append(transferfunction, [[i, fourier_transform[fr], trans, (trans-fourier_transform[fr])]],0)
np.save('full transferfunction.npy',transferfunction)



q1 = plt.quiver(0,0,np.real(transferfunction[0,1]),np.imag(transferfunction[0,1]), angles='xy', scale_units='xy', scale=1, color = 'b')
q2 = plt.quiver(0,0,np.real(transferfunction[0,2]),np.imag(transferfunction[0,2]), angles='xy', scale_units='xy', scale=1, color='r')
q3 = plt.quiver(np.real(transferfunction[0,1]),np.imag(transferfunction[0,1]),np.real(transferfunction[0,3]),np.imag(transferfunction[0,3]), angles='xy', scale_units='xy', scale=1, color = 'g')
plt.show()
# for q in range(51):
#     q1.set_UVC(np.real(transferfunction[q,0]),np.imag(transferfunction[q,0]))
#     q2.set_UVC(np.real(transferfunction[q,1]),np.imag(transferfunction[q,1]))
#     q3.set_offsets((np.real(transferfunction[q,0]),np.imag(transferfunction[q,0])))
#     q3.set_UVC(np.real(transfromfunction[q]),np.imag(transfromfunction[q]))

    
#     plt.pause(.5)

In [117]:
fs = np.arange(0.5,3.05,.05)

amplitude = np.abs(np.load('full transferfunction.npy'))
phase = np.rad2deg(np.angle(np.load('full transferfunction.npy')))


fig4, (bode_amplitude, bode_phase) = plt.subplots(2,1,sharex=True)
fig4_manager = plt.get_current_fig_manager()
fig4_manager.set_window_title('bodeplot East Z')
fig4_manager.window.showMaximized()

bode_amplitude.set_xscale('log'); bode_amplitude.set_yscale('log')
bode_amplitude.set_ylabel('Magnitude $|H(s)|$')
bode_phase.set_ylabel('fase $arg(H(s))$')
bode_amplitude.grid(True,'both')
bode_phase.grid(True, 'both')

bode_amplitude.plot(fs, amplitude[:,-1])
bode_phase.plot(fs, phase[:,-1])

bode_amplitude.axvline(1.4)

fig4.show()
plt.pause(0.002)
fig4.tight_layout()
